# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 8 2022 12:00AM,239033,10,0085741075,ISDIN Corporation,Released
1,Jun 8 2022 12:00AM,239039,10,0085741583,ISDIN Corporation,Released
2,Jun 8 2022 12:00AM,239039,10,0085741584,ISDIN Corporation,Released
3,Jun 8 2022 12:00AM,239033,10,0085744192,ISDIN Corporation,Released
4,Jun 8 2022 12:00AM,239033,10,0085744232,ISDIN Corporation,Released
5,Jun 8 2022 12:00AM,239033,10,0085744254,ISDIN Corporation,Released
6,Jun 8 2022 12:00AM,239033,10,0085744288,ISDIN Corporation,Released
7,Jun 8 2022 12:00AM,239033,10,0085744289,ISDIN Corporation,Released
8,Jun 8 2022 12:00AM,239033,10,0085744290,ISDIN Corporation,Released
9,Jun 8 2022 12:00AM,239033,10,0085744317,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
30,239039,Released,2
31,239041,Released,1
32,239042,Released,1
33,239052,Released,1
34,239053,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239039,NaN,NaN,2.0
239041,NaN,NaN,1.0
239042,NaN,NaN,1.0
239052,NaN,NaN,1.0
239053,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238942,0.0,6.0,11.0
238956,3.0,10.0,9.0
238958,0.0,0.0,7.0
238968,0.0,0.0,1.0
238984,0.0,1.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238942,0,6,11
238956,3,10,9
238958,0,0,7
238968,0,0,1
238984,0,1,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238942,0,6,11
1,238956,3,10,9
2,238958,0,0,7
3,238968,0,0,1
4,238984,0,1,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238942,,6,11
1,238956,3,10,9
2,238958,,,7
3,238968,,,1
4,238984,,1,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 8 2022 12:00AM,239033,10,ISDIN Corporation
1,Jun 8 2022 12:00AM,239039,10,ISDIN Corporation
52,Jun 7 2022 5:22PM,239053,16,"SHL Pharma, LLC"
53,Jun 7 2022 5:19PM,239052,21,"NBTY Global, Inc."
54,Jun 7 2022 4:58PM,239042,21,"NBTY Global, Inc."
55,Jun 7 2022 4:57PM,239041,16,"SHL Pharma, LLC"
56,Jun 7 2022 4:03PM,239036,21,"NBTY Global, Inc."
57,Jun 7 2022 4:00PM,239035,19,"AdvaGen Pharma, Ltd"
64,Jun 7 2022 2:49PM,239020,10,"Nextsource Biotechnology, LLC"
66,Jun 7 2022 2:36PM,239019,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 8 2022 12:00AM,239033,10,ISDIN Corporation,,,50
1,Jun 8 2022 12:00AM,239039,10,ISDIN Corporation,,,2
2,Jun 7 2022 5:22PM,239053,16,"SHL Pharma, LLC",,,1
3,Jun 7 2022 5:19PM,239052,21,"NBTY Global, Inc.",,,1
4,Jun 7 2022 4:58PM,239042,21,"NBTY Global, Inc.",,,1
5,Jun 7 2022 4:57PM,239041,16,"SHL Pharma, LLC",,,1
6,Jun 7 2022 4:03PM,239036,21,"NBTY Global, Inc.",,,1
7,Jun 7 2022 4:00PM,239035,19,"AdvaGen Pharma, Ltd",,,7
8,Jun 7 2022 2:49PM,239020,10,"Nextsource Biotechnology, LLC",,,2
9,Jun 7 2022 2:36PM,239019,10,ISDIN Corporation,10,18,6


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 8 2022 12:00AM,239033,10,ISDIN Corporation,50,,
1,Jun 8 2022 12:00AM,239039,10,ISDIN Corporation,2,,
2,Jun 7 2022 5:22PM,239053,16,"SHL Pharma, LLC",1,,
3,Jun 7 2022 5:19PM,239052,21,"NBTY Global, Inc.",1,,
4,Jun 7 2022 4:58PM,239042,21,"NBTY Global, Inc.",1,,
5,Jun 7 2022 4:57PM,239041,16,"SHL Pharma, LLC",1,,
6,Jun 7 2022 4:03PM,239036,21,"NBTY Global, Inc.",1,,
7,Jun 7 2022 4:00PM,239035,19,"AdvaGen Pharma, Ltd",7,,
8,Jun 7 2022 2:49PM,239020,10,"Nextsource Biotechnology, LLC",2,,
9,Jun 7 2022 2:36PM,239019,10,ISDIN Corporation,6,18,10


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 8 2022 12:00AM,239033,10,ISDIN Corporation,50,,
1,Jun 8 2022 12:00AM,239039,10,ISDIN Corporation,2,,
2,Jun 7 2022 5:22PM,239053,16,"SHL Pharma, LLC",1,,
3,Jun 7 2022 5:19PM,239052,21,"NBTY Global, Inc.",1,,
4,Jun 7 2022 4:58PM,239042,21,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 8 2022 12:00AM,239033,10,ISDIN Corporation,50.0,NaN,NaN
1,Jun 8 2022 12:00AM,239039,10,ISDIN Corporation,2.0,NaN,NaN
2,Jun 7 2022 5:22PM,239053,16,"SHL Pharma, LLC",1.0,NaN,NaN
3,Jun 7 2022 5:19PM,239052,21,"NBTY Global, Inc.",1.0,NaN,NaN
4,Jun 7 2022 4:58PM,239042,21,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 8 2022 12:00AM,239033,10,ISDIN Corporation,50.0,0.0,0.0
1,Jun 8 2022 12:00AM,239039,10,ISDIN Corporation,2.0,0.0,0.0
2,Jun 7 2022 5:22PM,239053,16,"SHL Pharma, LLC",1.0,0.0,0.0
3,Jun 7 2022 5:19PM,239052,21,"NBTY Global, Inc.",1.0,0.0,0.0
4,Jun 7 2022 4:58PM,239042,21,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1673076,63.0,18.0,10.0
12,238984,1.0,1.0,0.0
16,478094,2.0,0.0,0.0
18,478026,2.0,0.0,0.0
19,717016,8.0,1.0,0.0
20,1911881,138.0,20.0,3.0
21,1195135,5.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1673076,63.0,18.0,10.0
1,12,238984,1.0,1.0,0.0
2,16,478094,2.0,0.0,0.0
3,18,478026,2.0,0.0,0.0
4,19,717016,8.0,1.0,0.0
5,20,1911881,138.0,20.0,3.0
6,21,1195135,5.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,63.0,18.0,10.0
1,12,1.0,1.0,0.0
2,16,2.0,0.0,0.0
3,18,2.0,0.0,0.0
4,19,8.0,1.0,0.0
5,20,138.0,20.0,3.0
6,21,5.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,63.0
1,12,Released,1.0
2,16,Released,2.0
3,18,Released,2.0
4,19,Released,8.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,18,19,20,21
Status,,,,,,,
Completed,10.0,0.0,0.0,0.0,0.0,3.0,0.0
Executing,18.0,1.0,0.0,0.0,1.0,20.0,0.0
Released,63.0,1.0,2.0,2.0,8.0,138.0,5.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,18,19,20,21
0,Completed,10.0,0.0,0.0,0.0,0.0,3.0,0.0
1,Executing,18.0,1.0,0.0,0.0,1.0,20.0,0.0
2,Released,63.0,1.0,2.0,2.0,8.0,138.0,5.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,18,19,20,21
0,Completed,10.0,0.0,0.0,0.0,0.0,3.0,0.0
1,Executing,18.0,1.0,0.0,0.0,1.0,20.0,0.0
2,Released,63.0,1.0,2.0,2.0,8.0,138.0,5.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()